# Simpson's paradox

## Example from Causality Primer

From [Primer](http://bayes.cs.ucla.edu/PRIMER/primer-ch1.pdf), table 1.1. is

Subpopulations | No Treatment | Treatment 
------------ | ------------- | ------
Male |  234 of 270 recover (87%) | 81 of 87 recover (93%) 
Female | 55 of 80 recover (69%) | 192 of 263 recover (73%)
Total | 289 of 350 (83%)) | 273 of 350 (78%)


We exchange the rows due to keys having to be in alpha order for the fake-data-for-learning package (see [this issue](https://github.com/munichpavel/fake-data-for-learning/issues/8)):

Subpopulations | No Treatment | Treatment 
------------ | ------------- | ------
Female | 55 of 80 recover (69%) | 192 of 263 recover (73%)
Male |  234 of 270 recover (87%) | 81 of 87 recover (93%) 
Total | 289 of 350 (83%)) | 273 of 350 (78%)



## Equations for Simpson paradox

Keeping with the above example, we consider the counts above as being derived from a space of counts along dimensions $(\mathrm{RECOVERED}, \mathrm{GENDER}, \mathrm{TREATED})$ of $\mathbb{N}^2 \times \mathbb{N}^2  \times \mathbb{N}^2$:

\begin{align}
n_{000} &= \textrm{Count of non-recovered females who received no treatment} \\
n_{100} &= \textrm{Count of recovered females who received no treatment} \\
n_{010} &= \textrm{Count of non-recovered males who received no treatment} \\
n_{110} &= \textrm{Count of recovered males who received no treatment} \\
n_{001} &= \textrm{Count of non-recovered females who received treatment} \\
n_{101} &= \textrm{Count of recovered females who received treatment} \\
n_{011} &= \textrm{Count of non-recovered males who received treatment} \\
n_{111} &= \textrm{Count of recovered males who received treatment}
\end{align}

To denote sums along one or more of the count dimensions of RECOVERED, GENDER, TREATED, we substitute the dimension along which we sum by a $+$. So $n_{00+}=147$ is the count of recovered females, regardless of treatment. Hence we can re-write the above table as 

Subpopulations | No Treatment | Treatment 
:----------: | :-----------: | :----: 
Female |  $n_{100}$ of $n_{+00}$ recover (ratio $\frac{n_{100}}{n_{+00}}$) | $n_{101}$ of $n_{+01}$ recover (ratio $\frac{n_{101}}{n_{+01}})$ 
Male | $n_{110}$ of $n_{+10}$  recover (ratio $\frac{n_{110}}{n_{+10}})$ | $n_{111}$ of $n_{+11}$ recover (ratio $\frac{n_{111}}{n_{+11}})$
Total |  $n_{1+0}$ of $n_{++0}$  recover $\left(\frac{n_{1+1}}{n_{++0}}\right)$ |  $n_{1+1}$ of $n_{++1}$  recover (ratio $\frac{n_{1+1}}{n_{++1}})$ 


## As contingency tables

For computation, the $n_{ijk}$ lend itself to representing experiment outcomes grouped by sub-populations as contigency tables; see [Lectures on Algebraic Statistics](http://math.berkeley.edu/~bernd/owl.pdf), Section 1.1]. 

In this example, the contingency table is a 2x2x2 matrices of event counts. Using [xarray](http://xarray.pydata.org/en/stable/), we can define this matrix in a convenient form for defining, accessing and performing operations (e.g. the marginal sums of the $+$ notation).

In [ ]:
import numpy as np
import xarray as xr

In [ ]:
U = [
    [
        [25, 71],
        [36, 6]
        
    ], 
    [
         [55, 192],
        [234, 81]
    ]
       
]
gender_vals = ['female', 'male']
treated_vals = [0, 1]
recovered_vals = [0, 1]
counts = xr.DataArray(
    U, 
    dims=('recovered', 'gender', 'treated'),
    coords=[recovered_vals, gender_vals, treated_vals]
)

print("Check against counts from contingency table above:\n")
events = [
    dict(recovered=1, gender='female', treated=0),
    dict(recovered=1, gender='female', treated=1),
    dict(recovered=1, gender='male'),
    dict(recovered=1, gender='male'),

]

for event in events:
    marginal_counts = counts.sel(**event)
    print(f'Counts for event(s) {event}: \n{marginal_counts.values}')

Then the statement that the subpopulations receiving treatment recover better becomes the inequalities

\begin{align}
\frac{P(R=1|G=0, T=1)}{P(R=1 | G=0, T=0)} > 1\\
\frac{P(R=1|G=1, T=1)}{P(R=1|G=1, T=0)} > 1 \label{eq:SPR} \tag{SPR} \\
\end{align}

which we call the *subpopulation odds-ratio* (SPR) inequalities.

### More examples

#### Equal recovery for one subpopulation

In the below example, men show no difference in recovery between the treatment and non-treatment groups, and women fare better with treatment, yet overall no-treatment has a higher recovery ratio.

Subpopulations | Treatment | No treatment
------------ | ------------- | ----------
Female | 22 of 30 recover (72%) | 7 of 10 recover (70%)
Male | 9 of 10 recover (90%) | 27 of 30 recover (90%)
Total | 31 of 40 (78%) | 34 of 40 recover (85%)

In a variation of the above example, the male and overall populations experience no difference between treatment and no-treatment groups, but the female population recovers at a much higher rate with treatment.

Subpopulations | Treatment | No treatment
------------ | ------------- | ----------
Female | 22 of 30 recover (72%) | 4 of 10 recover (40%)
Male | 9 of 10 recover (90%) | 27 of 30 recover (90%)
Total | 31 of 40 (78%) | 31 of 40 recover (78%)

## References

* JUDEA PEARL, MADELYN GLYMOUR, NICHOLAS P. JEWELL, [CAUSAL INFERENCE IN STATISTICS: A PRIMER](http://bayes.cs.ucla.edu/PRIMER/primer-ch1.pdf)
* Mathias Drton, Bernd Sturmfels, Seth Sullivant, [Lectures on Algebraic Statistics](http://math.berkeley.edu/~bernd/owl.pdf)
* Aleksandra B. Slavkovic, Stephen E. Fienberg, [Algebraic geometry of 2 × 2 contingency tables](http://personal.psu.edu/abs12/Research/Papers/slavfienPistone2009.pdf)

In [ ]:
U = [[
        [1, 3],
        [8, 6]
    
    ], [
        [9, 27],
        [22, 4]
    ]]
gender_vals = ['female', 'male']
treated_vals = [0, 1]
recovered_vals = [0, 1]
counts = xr.DataArray(
    U,
    dims=("recovered", "treated", "gender"),
    coords=[recovered_vals, treated_vals, gender_vals]
#     dims=("treated", "gender", "recovered"),
#     coords=[treated_vals, gender_vals, recovered_vals]
)

events = [
    dict(treated=0, gender='female', recovered=0),
    dict(treated=0, gender='female', recovered=1),
    dict(treated=0, gender='male'),
    dict(recovered=1)
]

for event in events:
    marginal_counts = counts.sel(**event)
    print(f'Counts for event(s) {event}: \n{marginal_counts.values}')

In [ ]:
U = [[
        [4, 6],
        [3, 27]
    
    ], [
        [8, 22],
        [1, 9]
    ]]
gender_vals = ['female', 'male']
treated_vals = [0, 1]
recovered_vals = [0, 1]
counts = xr.DataArray(
    U, 
    dims=("treated", "gender", "recovered"),
    coords=[treated_vals, gender_vals, recovered_vals]
)

events = [
    dict(treated=0, gender='female', recovered=0),
    dict(treated=0, gender='female', recovered=1),
    dict(treated=0, gender='male'),
    dict(recovered=1)
]

for event in events:
    marginal_counts = counts.sel(**event)
    print(f'Counts for event(s) {event}: \n{marginal_counts.values}')

## Exercise: No-simpson

Show that if $N_{ij}$ are all constant, then Simpson's paradox cannot occur.

## Exercise: More no-Simpson

Assuming that the treatment and non-treatment population are equal, show that if further $N_{00} = N_{01}$ and $N_{10} = N_{11}$, Simpson's paradox cannot occur.

## Exercise: Even more no-Simpson

Consider the case of a treatment in which all subpopulations react the same way, $p_{ijk} = \lambda_k$ for $k=0, 1$. Show that Simpson's paradox cannot occur in this case (specifically, that the treatment and no-treatment population have the same recovery ratio).

### Exercise: Even morer no-Simpson

Show that if $\lambda_{00} = \lambda_{10}$ and $\lambda_{01} = \lambda_{11}$ then Simpson's paradox cannot occur.

## Generating Simpson examples

Let us further restrict to one sub-population for which we want to maximize the Simpson's skew, and further that the e.g. assume

Maximize $\lambda_{10} - \lambda_{11} = \epsilon$ subject to

\begin{align}
\lambda_{00} - \lambda

## ... to conditional probability matrices ...

In [ ]:
cpt = counts / counts.sum(dim='recovered')
cpt

In [ ]:
cpt.sel(treated=0, gender='female')
cpt.sel(gender='female')

In [ ]:
counts.sel(gender='female').sum(dim=['recovered'])

### Exercise: independence of discrete probability distribution via determinants

Show that the variables $G, T, R$ are independent if and only if the matrix $(p_{ijk})$ has rank 1.

Are $G, T, R$ in the above (smaller) example independent?

In [ ]:
n = float(counts.sum())
p = counts / n
p

In [ ]:
np.linalg.matrix_rank(p)

In [ ]:
26/40

In [ ]:
p.sel(treated=0).sum()